# Interacting with CSVs

In [1]:
import os 
from dotenv import load_dotenv

load_dotenv()

#For Azure
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")
AZURE_DEPLOYMENT = os.getenv("AZURE_DEPLOYMENT")
AZURE_ENDPOINT = os.getenv("AZURE_ENDPOINT")

#For Openai
OPENAI_API_KEY= os.getenv("OPENAI_API_KEY")


In [2]:
from IPython.display import Markdown, HTML, display
from langchain.schema import HumanMessage
from langchain_openai import AzureChatOpenAI
import pandas as pd

In [11]:
# In case you use AzureopenAI
#model = AzureChatOpenAI(
#    openai_api_version=OPENAI_API_VERSION,
#    azure_deployment=AZURE_DEPLOYMENT,
#    azure_endpoint=AZURE_ENDPOINT,
#)

In [7]:
#For OpenAI directly
from langchain_openai import ChatOpenAI


model = ChatOpenAI(
    model_name="gpt-4",  
    openai_api_key=os.getenv("OPENAI_API_KEY"), 
)

messages = [
    ("system", "You are a helpful translator. Translate the user sentence to French."),
    ("human", "I love programming.")
]
response = model.invoke(messages)
print(response.content)

J'aime la programmation.


## Dataload

In [8]:
#os.makedirs("data",exist_ok=True)
#!wget https://covidtracking.com/data/download/all-states-history.csv -P ./data/
file_url = "./data/all-states-history.csv"
df = pd.read_csv(file_url).fillna(value = 0)

In [9]:
df.head()

,date,state,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
0,2021-03-07,AK,305.0,0.0,0,0.0,1293.0,1293.0,33.0,0,...,1731628.0,0,0.0,0.0,0.0,0.0,0.0,0,1731628.0,0
1,2021-03-07,AL,10148.0,7963.0,-1,2185.0,45976.0,45976.0,494.0,0,...,2323788.0,2347,0.0,0.0,119757.0,0.0,2323788.0,2347,0.0,0
2,2021-03-07,AR,5319.0,4308.0,22,1011.0,14926.0,14926.0,335.0,11,...,2736442.0,3380,0.0,0.0,0.0,481311.0,0.0,0,2736442.0,3380
3,2021-03-07,AS,0.0,0.0,0,0.0,0.0,0.0,0.0,0,...,2140.0,0,0.0,0.0,0.0,0.0,0.0,0,2140.0,0
4,2021-03-07,AZ,16328.0,14403.0,5,1925.0,57907.0,57907.0,963.0,44,...,7908105.0,45110,580569.0,0.0,444089.0,0.0,3842945.0,14856,7908105.0,45110


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20780 entries, 0 to 20779
Data columns (total 41 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   date                              20780 non-null  object 
 1   state                             20780 non-null  object 
 2   death                             20780 non-null  float64
 3   deathConfirmed                    20780 non-null  float64
 4   deathIncrease                     20780 non-null  int64  
 5   deathProbable                     20780 non-null  float64
 6   hospitalized                      20780 non-null  float64
 7   hospitalizedCumulative            20780 non-null  float64
 8   hospitalizedCurrently             20780 non-null  float64
 9   hospitalizedIncrease              20780 non-null  int64  
 10  inIcuCumulative                   20780 non-null  float64
 11  inIcuCurrently                    20780 non-null  float64
 12  nega

## Prepare the langchain dataframe agent:

In [10]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(llm=model,df=df,verbose=True, allow_dangerous_code=True)

agent.invoke("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: We can use the shape attribute of pandas dataframe to get the number of rows.
Action: python_repl_ast
Action Input: df.shape[0]20780I now know the final answer
Final Answer: There are 20780 rows in the dataframe.

> Finished chain.


{'input': 'how many rows are there?',
 'output': 'There are 20780 rows in the dataframe.'}

## Designing Prompts

In [13]:
CSV_PROMPT_PREFIX = """
First set the pandas display options to show all the columns,
get the column names, then answer the question.
"""

CSV_PROMPT_SUFFIX = """
- **ALWAYS** before giving the Final Answer, try another method.
Then reflect on the answers of the two methods you did and ask yourself
if it answers correctly the original question.
If you are not sure, try another method.
- If the methods tried do not give the same result,reflect and
try again until you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that
you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful
and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,
ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.
- **ALWAYS**, as part of your "Final Answer", explain how you got
to the answer on a section that starts with: "\n\nExplanation:\n".
In the explanation, mention the column names that you used to get
to the final answer.
"""

QUESTION = "How may patients were hospitalized during July 2020" 
"in Texas, and nationwide as the total of all states?"
"Use the hospitalizedIncrease column" 


agent.invoke(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX)



> Entering new AgentExecutor chain...
Thought: Because the date column includes dates in a string format, I need to convert these to datetime objects in order to filter out July 2020. Then, I can sum the 'hospitalizedIncrease' column for July 2020 to get the total number of patients hospitalized.

Action: python_repl_ast
Action Input: 
```python
import pandas as pd

# set the pandas display options to show all the columns
pd.set_option('display.max_columns', None)

# convert the 'date' column to datetime
df['date'] = pd.to_datetime(df['date'])

# filter the dataframe to only include rows from July 2020
july_2020 = df[(df['date'] >= '2020-07-01') & (df['date'] <= '2020-07-31')]

# sum the 'hospitalizedIncrease' column
total_hospitalized_july_2020 = july_2020['hospitalizedIncrease'].sum()
total_hospitalized_july_2020
```
63105The total number of patients hospitalized during July 2020 was calculated to be 63105 using the 'hospitalizedIncrease' column and filtering the dataframe to only 

{'input': '\nFirst set the pandas display options to show all the columns,\nget the column names, then answer the question.\nHow may patients were hospitalized during July 2020\n- **ALWAYS** before giving the Final Answer, try another method.\nThen reflect on the answers of the two methods you did and ask yourself\nif it answers correctly the original question.\nIf you are not sure, try another method.\n- If the methods tried do not give the same result,reflect and\ntry again until you have two methods that have the same result.\n- If you still cannot arrive to a consistent result, say that\nyou are not sure of the answer.\n- If you are sure of the correct answer, create a beautiful\nand thorough response using Markdown.\n- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,\nONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.\n- **ALWAYS**, as part of your "Final Answer", explain how you got\nto the answer on a section that starts with: "\n\nExplanation:\n".\nIn the explanation, 